## Event based

In [123]:
from cartopy import config
import cartopy
import cartopy.crs as ccrs
import climtas
import cmocean
import dask.array
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from scipy import stats
from scipy.stats import ttest_ind 
import seaborn as sns
import sparse
import xarray as xr

In [2]:
#Point of interest for heatwave events
lat = -23.25
lon = 113.5

In [3]:
# opening the input files with heatwave severity data 
thw = xr.open_dataset('/g/data/e14/cp3790/Charuni/Tasmania/aus-coastal-sev.nc').sel(latitude=lat, longitude=lon)
mhw = xr.open_dataset('/g/data/e14/cp3790/Charuni/Tasmania/aus-ocean-sev-2.nc').sel(latitude=lat, longitude=lon)

In [4]:
# using climtas.event.find_events to identify days where severity>1 for a minimum of 3 consecutive days 
thw_events = climtas.event.find_events(thw.severity > 1, min_duration = 3)

In [9]:
# Create a sparse array with (coords, values, shape)
event_duration_array = sparse.COO(thw_events['time'], thw_events['event_duration'], shape=thw.severity.shape)
event_duration_da = xr.DataArray(event_duration_array, coords=thw.severity.coords)
event_duration_da

Format,coo
Data Type,int64
Shape,"(13514,)"
nnz,171
Density,0.012653544472398994
Read-only,True
Size,2.7K
Storage ratio,0.0


In [10]:
def get_coords(da, events):
    # Convert the index values to coordinates
    coords = {}
    for d in da.dims:
        coords[d] = da[d].values[events[d].values]
    
    # Also work out when the event ends
    coords['time_end'] = da['time'].values[events['time'].values + events['event_duration'].values-1]
    #coords['event_duration'] = coords['time_end'] - coords['time'] 
    coords['event_duration'] = events['event_duration'].values
    
    return pd.DataFrame(coords, index=events.index)

In [11]:
thw_new_H = get_coords(event_duration_da, thw_events)
thw_new_H

,time,time_end,event_duration
0,1982-12-06,1982-12-08,3
1,1983-06-07,1983-06-11,5
2,1983-08-07,1983-08-10,4
3,1983-09-16,1983-09-21,6
4,1983-10-22,1983-10-25,4
...,...,...,...
166,2018-01-02,2018-01-06,5
167,2018-01-12,2018-01-14,3
168,2018-03-07,2018-03-11,5
169,2018-05-14,2018-05-16,3


In [12]:
# using climtas.event.find_events to identify days where severity>1 for a minimum of 5 consecutive days (MHW events)
mhw_events = climtas.event.find_events(mhw.severity > 1, min_duration = 5)

In [14]:
mhw_duration_array = sparse.COO(mhw_events['time'], mhw_events['event_duration'], shape=mhw.severity.shape)
mhw_duration_da = xr.DataArray(event_duration_array, coords=mhw.severity.coords)
mhw_duration_da

Format,coo
Data Type,int64
Shape,"(13514,)"
nnz,171
Density,0.012653544472398994
Read-only,True
Size,2.7K
Storage ratio,0.0


In [22]:
mhw_new = get_coords(mhw_duration_da, mhw_events)
mhw_new

,time,time_end,event_duration
0,1982-03-15,1982-03-20,6
1,1982-04-16,1982-04-20,5
2,1982-04-28,1982-05-10,13
3,1982-08-07,1982-08-11,5
4,1982-08-15,1982-08-19,5
...,...,...,...
96,2015-09-15,2015-10-17,33
97,2015-10-21,2015-10-30,10
98,2016-07-17,2016-07-21,5
99,2017-02-28,2017-03-06,7


In [23]:
mhw_new.rename(columns={"time": "time_mhw", "time_end": "time_end_mhw", "event_duration": "event_duration_mhw"}, inplace=True)
mhw_new

,time_mhw,time_end_mhw,event_duration_mhw
0,1982-03-15,1982-03-20,6
1,1982-04-16,1982-04-20,5
2,1982-04-28,1982-05-10,13
3,1982-08-07,1982-08-11,5
4,1982-08-15,1982-08-19,5
...,...,...,...
96,2015-09-15,2015-10-17,33
97,2015-10-21,2015-10-30,10
98,2016-07-17,2016-07-21,5
99,2017-02-28,2017-03-06,7


In [24]:
# concatenating the THW and MHW dataframes 
df_merge_col = pd.concat([thw_new_H, mhw_new], axis=1)
df_merge_col

,time,time_end,event_duration,time_mhw,time_end_mhw,event_duration_mhw
0,1982-12-06,1982-12-08,3,1982-03-15,1982-03-20,6.0
1,1983-06-07,1983-06-11,5,1982-04-16,1982-04-20,5.0
2,1983-08-07,1983-08-10,4,1982-04-28,1982-05-10,13.0
3,1983-09-16,1983-09-21,6,1982-08-07,1982-08-11,5.0
4,1983-10-22,1983-10-25,4,1982-08-15,1982-08-19,5.0
...,...,...,...,...,...,...
166,2018-01-02,2018-01-06,5,NaT,NaT,NaN
167,2018-01-12,2018-01-14,3,NaT,NaT,NaN
168,2018-03-07,2018-03-11,5,NaT,NaT,NaN
169,2018-05-14,2018-05-16,3,NaT,NaT,NaN


In [30]:
#Calculating the overlap between THW and MHW events 
from datetime import datetime
from collections import namedtuple
Range = namedtuple('Range', ['start', 'end'])
overlap = []
for n in range (df_merge_col.shape[0]):
    mhw = Range(start=df_merge_col['time_mhw'].iloc[n], end=df_merge_col['time_end_mhw'].iloc[n])
    thw = Range(start=df_merge_col['time'].iloc[n], end=df_merge_col['time_end'].iloc[n])
    latest_start = max(mhw.start, thw.start)
    earliest_end = min(mhw.end, thw.end)
    delta = (earliest_end - latest_start).days + 1
    b = max(0, delta)
    overlap.append(b)

In [32]:
mod_fd = df_merge_col.assign(overlap_days = overlap)
mod_fd

,time,time_end,event_duration,time_mhw,time_end_mhw,event_duration_mhw,overlap_days
0,1982-12-06,1982-12-08,3,1982-03-15,1982-03-20,6.0,0
1,1983-06-07,1983-06-11,5,1982-04-16,1982-04-20,5.0,0
2,1983-08-07,1983-08-10,4,1982-04-28,1982-05-10,13.0,0
3,1983-09-16,1983-09-21,6,1982-08-07,1982-08-11,5.0,0
4,1983-10-22,1983-10-25,4,1982-08-15,1982-08-19,5.0,0
...,...,...,...,...,...,...,...
166,2018-01-02,2018-01-06,5,NaT,NaT,NaN,0
167,2018-01-12,2018-01-14,3,NaT,NaT,NaN,0
168,2018-03-07,2018-03-11,5,NaT,NaT,NaN,0
169,2018-05-14,2018-05-16,3,NaT,NaT,NaN,0


In [84]:
#co-occurring events is where there is a minimum overlap of 1 day between THW and MHW events
co_events = mod_fd[mod_fd.overlap_days != 0].reset_index()
co_events

,index,time,time_end,event_duration,time_mhw,time_end_mhw,event_duration_mhw,overlap_days
0,15,1985-02-26,1985-03-04,7,1985-02-12,1985-03-04,21.0,7


In [85]:
del co_events['time_mhw']
del co_events['time_end_mhw']
del co_events['overlap_days']
del co_events['event_duration_mhw']

co_events

#there appears to be only one such event 

,index,time,time_end,event_duration
0,15,1985-02-26,1985-03-04,7


In [86]:
#Changing time format to days so that it can be aligned with the mslp data array 

co_events.time = int(((co_events.time - np.datetime64('1982-01-01T00:00:00Z')) / np.timedelta64(1, 'h'))/24)
co_events.time_end = int(((co_events.time_end - np.datetime64('1982-01-01T00:00:00Z')) / np.timedelta64(1, 'h'))/24)
co_events

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-20.07/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  This is separate from the ipykernel package so we can avoid doing imports until
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-20.07/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  after removing the cwd from sys.path.


,index,time,time_end,event_duration
0,15,1152,1158,7


In [35]:
ds = xr.open_mfdataset('/g/data/e14/cp3790/Charuni/ERA5-MSL/era5_dailymsl_*.nc').sel(longitude=slice(90, 180), latitude=slice(0, -60))
mslp = ds['dmsl']/100
mslp.attrs['units'] = 'hPa'
mslp = mslp.chunk({'time':-1, 'latitude': 1, 'longitude':1})
mslp.load()

<xarray.DataArray 'dmsl' (time: 13514, latitude: 241, longitude: 360)>
array([[[1010.91406, 1010.89154, 1010.8837 , ..., 1009.1627 ,
         1009.16406, 1009.1641 ],
        [1010.90594, 1010.8862 , 1010.88007, ..., 1009.1615 ,
         1009.1632 , 1009.1599 ],
        [1010.8909 , 1010.87213, 1010.8641 , ..., 1009.16064,
         1009.15564, 1009.1555 ],
        ...,
        [ 988.88495,  988.79474,  988.7021 , ...,  985.26447,
          984.9652 ,  984.6678 ],
        [ 988.4409 ,  988.3532 ,  988.2565 , ...,  984.9257 ,
          984.6231 ,  984.31866],
        [ 987.97   ,  987.8854 ,  987.8    , ...,  984.6119 ,
          984.3041 ,  983.99664]],

       [[1010.4551 , 1010.41943, 1010.3959 , ..., 1009.4978 ,
         1009.4888 , 1009.4904 ],
        [1010.4494 , 1010.41656, 1010.396  , ..., 1009.50275,
         1009.4957 , 1009.4835 ],
        [1010.43787, 1010.4121 , 1010.3896 , ..., 1009.5096 ,
         1009.49835, 1009.4852 ],
...
        [ 988.6037 ,  988.2823 ,  987.9585 , ...,  983.33374,
          983.1482 ,  982.9578 ],
        [ 988.70526,  988.3979 ,  988.0821 , ...,  983.5454 ,
          983.3732 ,  983.2046 ],
        [ 988.79095,  988.49274,  988.1972 , ...,  983.7012 ,
          983.5415 ,  983.38226]],

       [[1013.5644 , 1013.55853, 1013.56085, ..., 1005.09   ,
         1005.0806 , 1005.07275],
        [1013.55554, 1013.5522 , 1013.56146, ..., 1005.0609 ,
         1005.05304, 1005.0402 ],
        [1013.54724, 1013.55194, 1013.56323, ..., 1005.0325 ,
         1005.02655, 1005.0174 ],
        ...,
        [1000.096  ,  999.9419 ,  999.78436, ...,  998.11255,
          998.0242 ,  997.9321 ],
        [ 999.77716,  999.6256 ,  999.4646 , ...,  997.93134,
          997.8518 ,  997.7753 ],
        [ 999.4197 ,  999.27185,  999.12286, ...,  997.79333,
          997.7246 ,  997.65625]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 1982-01-01 1982-01-02 ... 2018-12-31
  * longitude  (longitude) float32 90.0 90.25 90.5 90.75 ... 179.25 179.5 179.75
  * latitude   (latitude) float32 0.0 -0.25 -0.5 -0.75 ... -59.5 -59.75 -60.0
Attributes:
    units:    hPa

In [75]:
mslp_stack = mslp.stack(cell=('latitude', 'longitude'))

In [171]:
# Creating a new data array into which I subsequently save the mslp values
comp_co = xr.DataArray(np.random.randint(0, 100, size=(86760)),dims=["ncell"], coords=[mslp_stack.cell])
# 241*360=86760

In [172]:
# Adding an additional dimension to include event number
comp_co = comp_co.expand_dims({'event':1})

In [173]:
# I am trying to save the means calculated using climtas.event.map_events in the newly created data array comp_co

for x in mslp_stack.cell:
    mslp_ = mslp_stack.sel(cell=x)
    mslp_composite = climtas.event.map_events(mslp_, co_events, lambda x: x.mean().item()) #this gives the means of individual events at a grid cell
    #event_mean_mslp = mslp_composite.mean() #this gives the mean of all means 
    comp_co.loc[{"ncell":x, "event":x}] = mslp_composite

TypeError: invalid indexer array, does not have integer dtype: array([ 0., 90.])

In [111]:
# My expectation would be for the comp_co_da to have an 'event' dimension (in addition to latitude and longitude) 
# This is so that the t test could be done along the 'event' dimension 
comp_co_da = comp_co.unstack("ncell")
comp_co_da

<xarray.DataArray (latitude: 241, longitude: 360)>
array([[1010, 1010, 1010, ..., 1005, 1005, 1005],
       [1010, 1010, 1010, ..., 1005, 1005, 1005],
       [1010, 1010, 1010, ..., 1005, 1005, 1005],
       ...,
       [ 977,  977,  977, ..., 1006, 1006, 1007],
       [ 977,  977,  977, ..., 1006, 1006, 1006],
       [ 977,  977,  976, ..., 1005, 1006, 1006]])
Coordinates:
  * latitude   (latitude) float64 0.0 -0.25 -0.5 -0.75 ... -59.5 -59.75 -60.0
  * longitude  (longitude) float64 90.0 90.25 90.5 90.75 ... 179.2 179.5 179.8

In [93]:
#stand-alone events is where there NO overlap between THW and MHW events
stand_alone = mod_fd[mod_fd.overlap_days == 0].reset_index()
stand_alone

,index,time,time_end,event_duration,time_mhw,time_end_mhw,event_duration_mhw,overlap_days
0,0,1982-12-06,1982-12-08,3,1982-03-15,1982-03-20,6.0,0
1,1,1983-06-07,1983-06-11,5,1982-04-16,1982-04-20,5.0,0
2,2,1983-08-07,1983-08-10,4,1982-04-28,1982-05-10,13.0,0
3,3,1983-09-16,1983-09-21,6,1982-08-07,1982-08-11,5.0,0
4,4,1983-10-22,1983-10-25,4,1982-08-15,1982-08-19,5.0,0
...,...,...,...,...,...,...,...,...
165,166,2018-01-02,2018-01-06,5,NaT,NaT,NaN,0
166,167,2018-01-12,2018-01-14,3,NaT,NaT,NaN,0
167,168,2018-03-07,2018-03-11,5,NaT,NaT,NaN,0
168,169,2018-05-14,2018-05-16,3,NaT,NaT,NaN,0


In [94]:
del stand_alone['time_mhw']
del stand_alone['time_end_mhw']
del stand_alone['overlap_days']
del stand_alone['event_duration_mhw']

stand_alone

,index,time,time_end,event_duration
0,0,1982-12-06,1982-12-08,3
1,1,1983-06-07,1983-06-11,5
2,2,1983-08-07,1983-08-10,4
3,3,1983-09-16,1983-09-21,6
4,4,1983-10-22,1983-10-25,4
...,...,...,...,...
165,166,2018-01-02,2018-01-06,5
166,167,2018-01-12,2018-01-14,3
167,168,2018-03-07,2018-03-11,5
168,169,2018-05-14,2018-05-16,3


In [104]:
#Changing time format to days so that it can be aligned with the mslp data array 

stand_alone.time = (((stand_alone.time - np.datetime64('1982-01-01T00:00:00Z')) / np.timedelta64(1, 'h'))/24).astype(int)
stand_alone.time_end = (((stand_alone.time_end - np.datetime64('1982-01-01T00:00:00Z')) / np.timedelta64(1, 'h'))/24).astype(int)
stand_alone

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-20.07/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  This is separate from the ipykernel package so we can avoid doing imports until
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-20.07/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  after removing the cwd from sys.path.


,index,time,time_end,event_duration
0,0,339,341,3
1,1,522,526,5
2,2,583,586,4
3,3,623,628,6
4,4,659,662,4
...,...,...,...,...
165,166,13150,13154,5
166,167,13160,13162,3
167,168,13214,13218,5
168,169,13282,13284,3


In [174]:
# Creating a new data array into which I subsequently save the mslp values
comp = xr.DataArray(np.random.randint(0, 100, size=(86760)),dims=["ncell"], coords=[mslp_stack.cell])
# 241*360=86760

In [175]:
comp = comp.expand_dims({'event':170})

In [176]:
for x in mslp_stack.cell:
    mslp_ = mslp_stack.sel(cell=x)
    mslp_composite = climtas.event.map_events(mslp_, stand_alone, lambda x: x.mean().item()) #this gives the means of individual events at a grid cell
    #event_mean_mslp = mslp_composite.mean() #this gives the mean of all means 
    comp.loc[{"ncell":x, "event":x}] = mslp_composite

TypeError: invalid indexer array, does not have integer dtype: array([ 0., 90.])

In [106]:
comp_da = comp.unstack("ncell")
comp_da

<xarray.DataArray (latitude: 241, longitude: 360)>
array([[1009, 1009, 1009, ..., 1008, 1008, 1008],
       [1010, 1009, 1009, ..., 1008, 1008, 1008],
       [1009, 1009, 1009, ..., 1008, 1008, 1008],
       ...,
       [ 983,  983,  983, ...,  993,  993,  993],
       [ 983,  983,  983, ...,  992,  992,  992],
       [ 983,  983,  983, ...,  992,  992,  992]])
Coordinates:
  * latitude   (latitude) float64 0.0 -0.25 -0.5 -0.75 ... -59.5 -59.75 -60.0
  * longitude  (longitude) float64 90.0 90.25 90.5 90.75 ... 179.2 179.5 179.8

In [114]:
diff = comp_co_da - comp_da

In [137]:
# run the Welch's T-Test
r = stats.ttest_ind(comp_co_da, comp_da, axis=None, equal_var=False)

# Convert the p-values into a dataarray:
da = xr.DataArray(r[1], dims=('latitude', 'longitude'), coords={
    'latitude': comp_da['latitude'],
    'longitude': comp_da['longitude']
})